## IT Support Analyst

Ask ChatGPT to act as a IT support agent, process each customer IT ticket message and output the response in JSON with the following fields

```
orig_msg: The original customer message
orig_lang: Detected language of the customer message e.g. Spanish
category: 1-2 word describing the category of the problem
trans_msg: Translated customer message in English
response: Response to the customer in orig_lang
trans_response: Response to the customer in English
```

Try to use a JSON parser to get the responses in JSON for each ticket

In [6]:
import os
import yaml
with open('chatgpt-api-credentials.yml') as file:
    openai_key = yaml.safe_load(file)
os.environ['OPENAI_API_KEY'] = openai_key['OPENAI-API-KEY']

In [7]:
from langchain_openai import ChatOpenAI
chatgpt = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# Define your desired data structure - like a python data class.
class ITSupportResponse(BaseModel):
    orig_msg: str = Field(description="The original customer IT support query message")
    orig_lang: str = Field(description="Detected language of the customer message e.g. Spanish")
    category: str = Field(description="1-2 word describing the category of the problem")
    trans_msg: str = Field(description="Translated customer IT support query message in English")
    response: str = Field(description="Response to the customer in their original language - orig_lang")
    trans_response: str = Field(description="Response to the customer in English")


parser = JsonOutputParser(pydantic_object=ITSupportResponse)

In [2]:
prompt_txt = """
             Act as an Information Technology (IT) customer support agent.
             For the IT support message mentioned below
             use the following output format when generating the output response

             Output format instructions:
             {format_instructions}

             Customer IT support message:
             {it_support_msg}
             """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["it_support_msg"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [8]:
llm_chain = (prompt
              |
            chatgpt
              |
            parser)

In [11]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

In [14]:
responses = llm_chain.map().invoke([{"it_support_msg":msg} for msg in it_support_queue])

In [15]:
import pandas as pd

df = pd.DataFrame(responses)
df

,orig_msg,orig_lang,category,trans_msg,response,trans_response
0,Não consigo sincronizar meus contatos com o te...,Portuguese,Sync Issue,I can't sync my contacts with the phone. I alw...,Olá! Parece que você está enfrentando um probl...,Hello! It seems like you are experiencing an i...
1,Ho problemi a stampare i documenti da remoto. ...,Italian,Printing,I am having trouble printing documents remotel...,Grazie per averci contattato. Per risolvere il...,Thank you for reaching out to us. To resolve t...
2,プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。,Japanese,Printer,"I replaced the toner of the printer, but the p...",トナーを正しく交換したか確認してください。また、プリンターのメンテナンスを行うことで印刷品質...,Please check if you replaced the toner correct...
3,"Я не могу войти в систему учета времени, появл...",Russian,Login Issue,"I can't log into the time tracking system, an ...","Пожалуйста, попробуйте сбросить пароль и попро...",Please try resetting your password and attempt...
4,Internet bağlantım çok yavaş ve bazen tamamen ...,Turkish,Internet Connectivity,My internet connection is very slow and someti...,İnternet bağlantınızı kontrol etmek için aşağı...,You can follow the steps below to check your i...
5,Не могу установить обновление безопасности. По...,Russian,Installation,Unable to install security update. Error code ...,"Извините за возникшие проблемы. Пожалуйста, по...",Apologies for the inconvenience. Please try th...
